<a href="https://colab.research.google.com/github/evanzijianhe/Face_Recognition/blob/main/ECE595HW5_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install face_recognition

# Data Labels (Colab)

Because I did not find a good way to use text labels, I just converted them back to number labels based on the category file

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cvxpy as cp
import csv
from tabulate import tabulate
# import pandas
import numpy as geek 
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 100

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
category_path = "/content/drive/MyDrive/Colab Notebooks/HW5_data/ece595ml-s2022/category.csv"
train_path = "/content/drive/MyDrive/Colab Notebooks/HW5_data/ece595ml-s2022/train.csv"

In [ ]:
category = pd.read_csv(category_path, header = 0)
train = pd.read_csv(train_path, header = 0)

In [ ]:
train_index = train['Index']
train_number_data = np.shape(train_index)[0]
category_index = category['Index']
category_number_data = np.shape(category_index)[0]
train_label = np.zeros(train_number_data)

In [ ]:
for ii in range(category_number_data):
  category_index = category.iloc[ii,1]
  train_label_0 = train.loc[train['Category']== category_index ,'Index']
  train_label_vec = train_label_0.values
  for jj in range(len(train_label_vec)):
    train_label[train_label_vec[jj]]= int(ii)

In [ ]:
train_index_table = train_index.values
# train_small_label_table = {'index': train_small_index_table,'labels':train_small_label}
train_small_label_table = pd.DataFrame(data = {'index': train_index_table,'labels':train_label})

In [ ]:
pd.DataFrame.to_csv(train_small_label_table,'/content/drive/MyDrive/Colab Notebooks/HW5_data/ece595ml-s2022/train_category.csv')  

# Data Preparation (Local machine)

Capture Face from the picture, crop it and save it to local folder

In [ ]:
import numpy as np
import cv2
import os
import face_recognition
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

path = r'E:\train\facedata\test\test/'
image_list = os.listdir(path)

test_data = []


def save_file(img,name,x,y,width,height):
  image=img[y:y+height,x:x+width]
  cv2.imwrite(name,image)

def resize_pic(image, size=(100, 100)):
    height, width, _ = image.shape
    longest_edge = max(height, width)
    top, bottom, left, right = 0, 0, 0, 0
    if height < longest_edge:
        height_diff = longest_edge - height
        top = int(height_diff / 2)
        bottom = height_diff - top
    elif width < longest_edge:
        width_diff = longest_edge - width
        left = int(width_diff / 2)
        right = width_diff - left
    image_with_border = cv2.copyMakeBorder(image, top, bottom, left, \
                right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    resized_image = cv2.resize(image_with_border, size)
    return resized_image

def read_crop_save_image():
    number_img = len(image_list)
    CASE_PATH = r"E:\train\facedata\haarcascade_frontalface_default.xml"
    face_cascade=cv2.CascadeClassifier(CASE_PATH)
    for ii in range (number_img):
        img = face_recognition.load_image_file(path + '%s.jpg'%ii)
        plt.imshow = img
        gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray, 1.3, 5)
        faces = face_cascade.detectMultiScale(gray,scaleFactor=1.3,\
                          minNeighbors=5,minSize=(5,5),)
        if len(faces) == 0:
            cv2.imwrite(r'E:\train\facedata\test_data_face_crop\%s.bmp'%ii,img)
            print(ii)
        else:
            for (x,y,w,h) in faces:
              cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
              save_file(img,'%s%d.bmp'%(r'E:\train\facedata\test_data_face_crop/',ii),x,y,w,h)
if __name__ == '__main__':
    image_size = 100
    read_crop_save_image()

Read the face_cropped images and save it as .npy file. This file will be uploaded to colab and used to train prediction model 

In [ ]:
path = r'E:\train\facedata\test_data_face_crop/'
image_list = os.listdir(path)

test_data = []

def resize_pic(image, size=(100, 100)):
    height, width, _ = image.shape
    longest_edge = max(height, width)
    top, bottom, left, right = 0, 0, 0, 0
    if height < longest_edge:
        height_diff = longest_edge - height
        top = int(height_diff / 2)
        bottom = height_diff - top
    elif width < longest_edge:
        width_diff = longest_edge - width
        left = int(width_diff / 2)
        right = width_diff - left
    image_with_border = cv2.copyMakeBorder(image, top, bottom, left,\
                right, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    resized_image = cv2.resize(image_with_border, size)
    return resized_image


def read_image(size=None):
    data_x = []
    number_img = len(image_list)
    for i in range(number_img):
        try:
            im = face_recognition.load_image_file(path+'%s.bmp'%i)
            if size is None:
                size = (100, 100)
            im = resize_pic(im, size)
            data_x.append(np.asarray(im, dtype=np.int8))
            print(i)
        except IOError as e:
            print(e)
        except:
            print('UnknownError!')
    return data_x

if __name__ == '__main__':
    image_size = 100
    test_images = read_image(size = (image_size,image_size))
    test_images = np.asarray(test_images,dtype=np.float32)
    print(np.shape(test_images))
    np.save('test_data_face_crop',test_images)
    print('complete')


# Training Model (Colab)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
train_labels = np.load("/content/drive/MyDrive/Colab Notebooks/HW5_data/train_labels_array.npy")
train_input = np.load("/content/drive/MyDrive/Colab Notebooks/HW5_data/train_data_face_crop.npy")

In [ ]:
from keras.utils import np_utils
train_output = np_utils.to_categorical(train_labels)

In [ ]:
train_input /= 255.0
# valid_input /= 255.0

In [ ]:
from tensorflow import keras
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten
from keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD

In [ ]:
image_size = 100

In [ ]:
face_recognition_model=keras.Sequential()
face_recognition_model.add(Conv2D(32,(3,3),padding='valid',strides=(1,1),\
          input_shape=(image_size,image_size,3),activation='relu'))
face_recognition_model.add(Conv2D(32,(3,3),padding='valid',strides=(1,1),\
                              activation='relu'))

In [ ]:
face_recognition_model.add(MaxPooling2D(pool_size=(2, 2)))

In [ ]:
face_recognition_model.add(Dropout(0.2))

In [ ]:
face_recognition_model.add(Conv2D(64,(3,3),padding='valid',
strides=(1,1),
activation='relu'))
 
face_recognition_model.add(Conv2D(64,(3,3),padding='valid',
strides=(1,1),
activation='relu'))
 
face_recognition_model.add(MaxPooling2D(pool_size=(2,2)))
face_recognition_model.add(Dropout(0.2))

In [ ]:
face_recognition_model.add(Flatten())

In [ ]:
face_recognition_model.add(Dense(512, activation = 'relu',\
                  kernel_regularizer=l2(0.001)))
face_recognition_model.add(Dropout(0.4))

In [ ]:
face_recognition_model.add(Dense(len(train_output[0]),\
                    activation = 'softmax'))

In [ ]:
face_recognition_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 96, 96, 32)        9248      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 48, 48, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 48, 48, 32)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 46, 46, 64)        18496     
                                                                 
 conv2d_3 (Conv2D)           (None, 44, 44, 64)        36928     
                                                        

In [ ]:
learning_rate=0.0005
decay=1e-6
momentum=0.8
nesterov=True
sgd_optimizer=SGD(learning_rate=learning_rate,decay=decay,
momentum=momentum,nesterov=nesterov)

In [ ]:
opt = Adam(learning_rate=0.00001) 

In [ ]:
face_recognition_model.compile(loss='categorical_crossentropy',optimizer=opt,
metrics=['accuracy'])

In [ ]:
batch_size=20
epochs=100
face_recognition_model.fit(train_input,train_output,batch_size=batch_size,\
                epochs = epochs, validation_split = 0.3)

Epoch 1/100
2434/2434 [==============================] - 67s 23ms/step - loss: 5.3271 - accuracy: 0.0218 - val_loss: 5.0613 - val_accuracy: 0.0525
Epoch 2/100
2434/2434 [==============================] - 54s 22ms/step - loss: 4.8989 - accuracy: 0.0589 - val_loss: 4.6992 - val_accuracy: 0.1067
Epoch 3/100
2434/2434 [==============================] - 55s 22ms/step - loss: 4.6132 - accuracy: 0.0982 - val_loss: 4.4489 - val_accuracy: 0.1403
Epoch 4/100
2434/2434 [==============================] - 56s 23ms/step - loss: 4.4036 - accuracy: 0.1352 - val_loss: 4.2709 - val_accuracy: 0.1759
Epoch 5/100
2434/2434 [==============================] - 56s 23ms/step - loss: 4.2284 - accuracy: 0.1694 - val_loss: 4.1394 - val_accuracy: 0.2078
Epoch 6/100
2434/2434 [==============================] - 56s 23ms/step - loss: 4.0743 - accuracy: 0.1999 - val_loss: 4.0323 - val_accuracy: 0.2301
Epoch 7/100
2434/2434 [==============================] - 56s 23ms/step - loss: 3.9390 - accuracy: 0.2286 - val_loss: 3

In [ ]:
MODEL_PATH='/content/drive/MyDrive/Colab Notebooks/HW5_data/Training Result/face_model_train_small_12trial.h5'
face_recognition_model.save(MODEL_PATH)

# Face Recognition (Colab)

In [ ]:
import cv2
import os
import numpy as np
import keras
from keras.models import load_model
import face_recognition

In [ ]:
def resize_pic(image,size=(100,100)):
  height,width,_=image.shape
  longest_edge=max(height,width)
  top,bottom,left,right=0,0,0,0
  if height<longest_edge:
    height_diff=longest_edge-height
    top=int(height_diff/2)
    bottom=height_diff-top
  elif width<longest_edge:
    width_diff=longest_edge-width
    left=int(width_diff/2)
    right=width_diff-left

  image_with_border=cv2.copyMakeBorder(image,top,bottom,left,right,cv2.BORDER_CONSTANT,value = [0,0,0])

  resized_image=cv2.resize(image_with_border,size)

  return resized_image

In [ ]:
face_recognition_model = keras.Sequential()
MODEL_PATH = '/content/drive/MyDrive/Colab Notebooks/HW5_data/Training Result/face_model_train_small_12trial.h5'
face_recognition_model = load_model(MODEL_PATH)

In [ ]:
test_input = np.load("/content/drive/MyDrive/Colab Notebooks/HW5_data/test_data_face_crop.npy")

In [ ]:
predict_class = []
test_number = []
len_test = test_input.shape[0]
for idx in range (len_test):
  yhat = face_recognition_model.predict(test_input[idx:(idx+1)])
  yhat_array = np.rint(yhat)
  predict = np.argmax(yhat_array)
  test_number.append(idx)
  print(idx)
  predict_class.append(predict)

In [ ]:
import pandas as pd
pred_table = pd.DataFrame(data = {'test_number':test_number,'predict':predict_class})

In [ ]:
category_path = "/content/drive/MyDrive/Colab Notebooks/HW5_data/ece595ml-s2022/category.csv"
category = pd.read_csv(category_path, header = 0)

In [ ]:
predict_class_number = pred_table['test_number'].shape[0]

In [ ]:
predict_name_array = []
for ii in range(predict_class_number):
  predict_index = pred_table['test_number'][ii]
  predict_name = category.loc[category['Index'] == pred_table['predict'][ii],'Category'].values
  predict_name_0 = predict_name
  predict_name_array.append(predict_name_0)

In [ ]:
pred_table_name = pd.DataFrame(data = {'test_number':test_number,'predict':predict_class,'name':predict_name_array})

In [ ]:
pred_table_name['name'] = pred_table_name['name'].str.get(0)

In [ ]:
pred_table_name

,test_number,predict,name
0,0,26,Adriana Barraza
1,1,66,Albert Finney
2,2,52,Adrien Brody
3,3,26,Adriana Barraza
4,4,72,Albert Brooks
...,...,...,...
4972,4972,48,Andrew Lincoln
4973,4973,35,Abigail Breslin
4974,4974,95,Bill Hader
4975,4975,25,Amanda Crew


In [ ]:
pd.DataFrame.to_csv(pred_table_name,'/content/drive/MyDrive/Colab Notebooks/HW5_data/ece595ml-s2022/submission_9.csv') 

# Output PDF

In [ ]:
%%capture
!wget -nc https://raw.githubusercontent.com/brpy/colab-pdf/master/colab_pdf.py
from colab_pdf import colab_pdf
colab_pdf('ECE595HW5_Code.ipynb')